# Spatial Functions - overlay operations

## Preparing the data

First we need to load all the data clean it up and create buffers.  There is nothing that we haven't seen in this code block. We are just accumulating it all in one place.

In [ ]:
%matplotlib inline
import geopandas as gpd

# Read in all the raw data
buowl = gpd.read_file("data/BUOWL_Habitat.shp")
raptor = gpd.read_file("data/Raptor_Nests.shp")
linear = gpd.read_file("data/Linear_Projects.shp")
eagle = gpd.read_file("data/BAEA_Nests.shp")
counties = gpd.read_file("data/colorado_counties.shp", mask=eagle)

# Clean raptor data - get a subset that excludes 3 outlier points
raptor = raptor.cx[:-104.3, 39.5:40.6]

# Convert raw data to UTM NAD83/Zone 13 CRS (EPSG:26913)
buowl.to_crs(epsg=26913, inplace=True)
raptor.to_crs(epsg=26913, inplace=True)
linear.to_crs(epsg=26913, inplace=True)
eagle.to_crs(epsg=26913, inplace=True)
counties.to_crs(epsg=26913, inplace=True)

# Clean raptor data - relace coded field values
raptor['recentspec'] = raptor['recentspec'].str.replace('SWHA', 'Swainsons Hawk')
raptor['recentspec'] = raptor['recentspec'].str.replace('RTHA', 'Red-tail Hawk')
print("recentspec values = {}".format(raptor['recentspec'].unique()))

# add buff_dist field to raptors GeoDataFrame
species_buffer = {"Swainsons Hawk":333, "Red-tail Hawk":667, "Northern Harrier":500}
raptor['buf_dist']=raptor['recentspec'].map(species_buffer)

# add length field to linears GeoDataFrame
linear['length_m'] = linear['geometry'].length

# Create buffers
eagle['buffer'] = eagle['geometry'].buffer(804.5)
buowl['buffer'] = buowl['geometry'].buffer(300)
linear['buffer'] = linear['geometry'].buffer(linear['row_width'])
raptor['buffer'] = raptor['geometry'].buffer(raptor['buf_dist'])
                                 
eagle.set_geometry('buffer', inplace=True)
buowl.set_geometry('buffer', inplace=True)
linear.set_geometry('buffer', inplace=True)
raptor.set_geometry('buffer', inplace=True)

In [ ]:
raptor

## Simple plotting of more than 1 GeoDataFrame

So far in this course we have done some simple plotting but only of a single GeoDataFrame.  Here we will expand this to include maps with more than one GeoDataFrame.

As a result we will also be more specific about which colors we will use.

The key is that calling the plot function has a return value and we can use that return value to use as the basemap in the next call to the plot method using the ax parameter


In [ ]:
basemap = eagle.cx[490000:520000, 4430000:4460000].plot(color='darkgreen', alpha = 0.5, figsize=(15, 15))
basemap = raptor.cx[490000:520000, 4430000:4460000].plot(ax=basemap, color='darkblue', alpha = 0.5)
basemap = buowl.cx[490000:520000, 4430000:4460000].plot(ax=basemap, color='orangered', alpha = 0.5)
basemap = linear.cx[490000:520000, 4430000:4460000].plot(ax=basemap, cmap='tab10', column='type', legend=True)

In [ ]:
linear.cx[490000:520000, 4430000:4460000].sort_values('length_m', ascending=False).head()

In [ ]:
basemap = eagle.cx[505000:520000, 4440000:4455000].plot(color='darkgreen', alpha = 0.5, figsize=(15, 15))
basemap = raptor.cx[505000:520000, 4440000:4455000].plot(ax=basemap, color='darkblue', alpha = 0.5)
basemap = buowl.cx[505000:520000, 4440000:4455000].plot(ax=basemap, color='orangered', alpha = 0.5)
basemap = linear[linear['Project']==296].plot(ax=basemap, color='red')

## Different uses of intersection

### intersects - spatial predicate

Spatial predicates return true or false depending if the relationship is true.

The following code cell returns the raptor nests whose buffers intersect the row for linear project 296.

In [ ]:
linear_296 = linear[linear['Project']==296].unary_union
impacted_raptors = raptor[raptor['buffer'].intersects(linear_296)]
impacted_raptors

In [ ]:
linear_296

### intersection - spatial operation

Returns the intersection of two geometries as a geometry.


In [ ]:
impacted_raptors = impacted_raptors.copy()
impacted_raptors['impacted_poly']=impacted_raptors['buffer'].intersection(linear_296)
impacted_raptors['impacted_area']=impacted_raptors['impacted_poly'].area/10000
impacted_raptors

In [ ]:
basemap = eagle.cx[505000:520000, 4440000:4455000].plot(color='darkgreen', alpha = 0.5, figsize=(15, 15))
basemap = raptor.cx[505000:520000, 4440000:4455000].plot(ax=basemap, color='darkblue', alpha = 0.5)
basemap = buowl.cx[505000:520000, 4440000:4455000].plot(ax=basemap, color='orangered', alpha = 0.5)
basemap = impacted_raptors.set_geometry('impacted_poly').plot(ax=basemap, color='red', edgecolor='k')

In [ ]:
impacted_raptors[impacted_raptors['recentstat']=='ACTIVE NEST']['impacted_area'].max()

### Overlay method with how='intersection'

This is the traditional intersection operation from desktop GIS. It creates an entirely new dataset with a new feature for every combination of raptor buffer and linear buffer.  Each feature will have ALL the fields from both layers.

In [ ]:
raptor_intersection = gpd.overlay(raptor, linear, how='intersection')
raptor_intersection['area_ha'] = raptor_intersection['geometry'].area/10000
raptor_intersection

In [ ]:
raptor = raptor[['Nest_ID', 'recentstat', 'recentspec', 'buffer']]
raptor

In [ ]:
linear = linear[['Project', 'type', 'length_m', 'buffer']]
linear

In [ ]:
raptor_intersection = gpd.overlay(raptor, linear, how='intersection')
raptor_intersection['area_ha'] = raptor_intersection['geometry'].area/10000
raptor_intersection

In [ ]:
basemap = eagle.cx[505000:520000, 4440000:4455000].plot(color='darkgreen', alpha = 0.5, figsize=(15, 15))
basemap = raptor.cx[505000:520000, 4440000:4455000].plot(ax=basemap, color='darkblue', alpha = 0.5)
basemap = buowl.cx[505000:520000, 4440000:4455000].plot(ax=basemap, color='orangered', alpha = 0.5)
basemap = linear.cx[505000:520000, 4440000:4455000].plot(ax=basemap, color='yellow', alpha = 0.5, edgecolor='grey')
basemap = raptor_intersection.cx[505000:520000, 4440000:4455000].plot(ax=basemap, color='red', edgecolor='k')

### Intersections with different geometry types

Although intersections are generally thought of as relationships between two polygon geometries, they can also occur between geometries of different dimensions.  

In this case the order in which the GeoDataFrames are used on the overlay function is important.  The output geometry type will be the same as the first GeoDataFrame in the overlay function.

Consider if we want to perform an intersection with the raptor nest buffers and the linear features.  Not the buffers of the linear features that would return a row polygon and allow you to calculate how many hectares of the row was impacted by each raptor nest but rather an intersection of the raptor nest buffers and the actual lines.  This would allow you to answer questions such as how many meters of a pipeline are impacted.

We've already overwritten the raptor and linear geodataframes to include just a few select fields and the buffer geometry so we no longer have the actual linear geometry in the linear GeoDataFrame.  Lets run the first codecell in this notebook again to start from scratch.

In [ ]:
linear

Now lets run the code we ran above to generate an intersection but we will use the original linear geometry of the linear GeoDataFrame.

First lets select just the important columns from each GeoDataFrame

In [ ]:
raptor = raptor[['Nest_ID', 'recentstat', 'recentspec', 'buffer']]
raptor

In [ ]:
linear = linear[['Project', 'type', 'length_m', 'geometry']]
linear

This looks good but there is a problem.  Even though the linear GeoDataFrame has a GeoSeries it comes across as a pandas dataframe rather than a GeoPandas GeoDataFrame.  This caused a problem that I didn't see until I actually tried to do the intersection.

In [ ]:
linear.info()

Fortunately it is quite easy to convert a Pandas dataframe to a GeoPanda's GeoDataFrame.  You just pass it to the GeoDataFrame constrctor method. SInce it already has a GeoSeries we don't need to do anything else.

In [ ]:
linear = gpd.GeoDataFrame(linear)
linear.info()

In [ ]:
linear.cx[:,4400000:4480000].plot(figsize=(10,10))

In [ ]:
raptor_intersection_linear = gpd.overlay(linear, raptor, how='intersection')
raptor_intersection_linear['length_intersection'] = raptor_intersection_linear['geometry'].length
raptor_intersection_linear.sort_values('Project')

In [ ]:
basemap = eagle.cx[505000:520000, 4440000:4455000].plot(color='darkgreen', alpha = 0.5, figsize=(15, 15))
basemap = raptor.cx[505000:520000, 4440000:4455000].plot(ax=basemap, color='darkblue', alpha = 0.5)
basemap = buowl.cx[505000:520000, 4440000:4455000].plot(ax=basemap, color='orangered', alpha = 0.5)
basemap = linear.cx[505000:520000, 4440000:4455000].plot(ax=basemap, color='green', alpha = 0.5, edgecolor='grey')
basemap = raptor_intersection_linear.cx[505000:520000, 4440000:4455000].plot(ax=basemap, color='red', edgecolor='k')

In [ ]:
raptor_intersection.to_file('data/intersections.gpkg', layer='raptor_buffer', driver='GPKG')
raptor_intersection_linear.to_file('data/intersections.gpkg', layer='raptor_linear', driver='GPKG')

In [ ]:
import fiona
fiona.listlayers('data/intersections.gpkg')